# Weak Supervision Pipeline - News from Folha de SP

This notebook is an assignment for the Datacentric AI (IMD3011) course at Instituto Metrópole Digital (IMD), taught by Professor Elias Jacob. The goal is to develop a weak supervision pipeline as part of the first unit's assessment.
 
Here, we will perform news classification using articles extracted from Folha de S.Paulo, leveraging a dataset available on Kaggle. The workflow will include data exploration, preprocessing, and modeling steps, with a focus on weak supervision techniques for automatic labeling and classification of news articles.

In [ ]:

#Download the dataset from Kaggle
!curl -L -o ./news-of-the-site-folhauol.zip\
  https://www.kaggle.com/api/v1/datasets/download/marlesson/news-of-the-site-folhauol

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  187M  100  187M    0     0   154M      0  0:00:01  0:00:01 --:--:--  212M


In [1]:
import snorkel
import pandas as pd
from snorkel.labeling import (
    LFAnalysis,
    PandasLFApplier,
    filter_unlabeled_dataframe,
    labeling_function,
)
from snorkel.labeling.model.label_model import LabelModel
from snorkel.utils import probs_to_preds

snorkel.__version__

'0.9.9'

EDA

In [2]:
df = pd.read_csv("./data/articles.csv", low_memory=False)

df.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [3]:
print(df.shape)

(167053, 6)


In [4]:
df.category.value_counts(normalize=True)

category
poder                           0.131826
colunas                         0.129432
mercado                         0.125529
esporte                         0.118106
mundo                           0.102542
cotidiano                       0.101567
ilustrada                       0.097843
opiniao                         0.027087
paineldoleitor                  0.024010
saopaulo                        0.023675
tec                             0.013529
tv                              0.012822
educacao                        0.012679
turismo                         0.011392
ilustrissima                    0.008446
ciencia                         0.007991
equilibrioesaude                0.007854
sobretudo                       0.006327
bbc                             0.005866
folhinha                        0.005244
empreendedorsocial              0.005034
comida                          0.004957
asmais                          0.003280
ambiente                        0.002939
seminar

In [5]:
filtered_categories = ["poder", "mercado", "esporte", "mundo"]
df = df[df['category'].isin(filtered_categories)]

print(df.shape)

(79852, 6)


In [ ]:
print(df.columns)

# Generate index column to use as a unique identifier
df = df.drop(columns=["date", "category", "subcategory", "link"]).reset_index()

Index(['title', 'text', 'date', 'category', 'subcategory', 'link'], dtype='object')


In [7]:
for row in df.sample(50, random_state=271828).itertuples():
    print(f"{row.title} - {row.text}")
    print("\n")

Lojistas apostam em pontos não tradicionais buscando novos clientes - Pontos de venda incomuns, fora dos shoppings ou tradicionais comércios de rua, têm sido a aposta de empresários para renovar a clientela e cortar custos em ano de economia fraca.  Denise Schneider, 35, decidiu abrir um consultório odontológico dentro de uma escola de ensino fundamental e médio de Alvorada (RS).  Ela conta que já era proprietária de uma unidade da franquia da Ortoplan em um ponto na rua, então procurou uma alternativa.  "Não queria fazer mais do mesmo. Então encontrei um bairro com uma escola com 800 alunos", diz Schneider, que investirá em campanha de marketing e de conscientização sobre saúde bucal para conquistar não só os jovens mas também seus pais.  Para reduzir os riscos de baixa demanda, Schneider escolheu uma escola com espaço que permite à clínica ter uma saída para a rua e atender a clientela externa.  O sócio-fundador e presidente da Ortoplan, Faisal Ismail, diz que a rede aposta em pontos

In [8]:
# Convert all characters in the 'text' column to lowercase
# This ensures uniformity in text data, which is useful for text processing tasks
df["text"] = df["text"].str.lower()

df

,index,title,text
0,0,"Lula diz que está 'lascado', mas que ainda tem...",com a possibilidade de uma condenação impedir ...
1,2,Três reportagens da Folha ganham Prêmio Petrob...,três reportagens da folha foram vencedoras do ...
2,4,CBSS inicia acordos com fintechs e quer 30% do...,"o cbss, banco da holding elopar dos sócios bra..."
3,5,"Em encontro, Bono pergunta a Macri sobre argen...","o vocalista da banda irlandesa u2, bono, fez u..."
4,6,"Posso sair do Brasil quando e como quiser, diz...",o italiano cesare battisti disse nesta segunda...
...,...,...,...
79847,167046,"Apoiado pelos irmãos Gomes, petista toma posse...",engenheiro agrônomo e servidor licenciado do i...
79848,167048,"Em cenário de crise, tucano Beto Richa assume ...",o tucano beto richa tinha tudo para começar se...
79849,167049,Filho supera senador Renan Calheiros e assume ...,o economista renan filho (pmdb) assume nesta q...
79850,167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",destaques da programação desta quinta-feira (1...


In [9]:
from utils.text import (
    remove_accented_characters,
    remove_excessive_spaces,
    remove_repeated_letters,
    remove_repeated_non_word_characters,
)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Create a text cleaning pipeline using scikit-learn's Pipeline
# Each step in the pipeline applies a specific text cleaning function

pipeline_clean_text = Pipeline(
    [
        # Step 1: Remove accented characters from the text
        ("remove_accented_characters", FunctionTransformer(remove_accented_characters)),
        # Step 2: Remove excessive spaces from the text
        ("remove_excessive_spaces", FunctionTransformer(remove_excessive_spaces)),
        # Step 3: Remove repeated letters from the text
        ("remove_repeated_letters", FunctionTransformer(remove_repeated_letters)),
        # Step 4: Remove repeated non-word characters (e.g., punctuation) from the text
        (
            "remove_repeated_non_word_characters",
            FunctionTransformer(remove_repeated_non_word_characters),
        ),
    ]
)

In [11]:
# Print the shape of the DataFrame before dropping rows with missing 'text' values
# This shows the number of rows and columns in the DataFrame initially
print(df.shape)

# Drop rows where the 'text' column has NaN (missing) values
# The 'inplace=True' parameter modifies the DataFrame in place without returning a new DataFrame
df = df.dropna(subset=["text"])

print(df.shape)

(79852, 3)
(79852, 3)


In [12]:
# Apply the text cleaning pipeline to the 'text' column of the DataFrame
# 'pipeline_clean_text.transform' applies all the cleaning steps defined in the pipeline
# The 'apply' method applies the transformation to each element in the 'text' column
df["text"] = df["text"].apply(pipeline_clean_text.transform)

df

,index,title,text
0,0,"Lula diz que está 'lascado', mas que ainda tem...",com a possibilidade de uma condenacao impedir ...
1,2,Três reportagens da Folha ganham Prêmio Petrob...,tres reportagens da folha foram vencedoras do ...
2,4,CBSS inicia acordos com fintechs e quer 30% do...,"o cbss, banco da holding elopar dos socios bra..."
3,5,"Em encontro, Bono pergunta a Macri sobre argen...","o vocalista da banda irlandesa u2, bono, fez u..."
4,6,"Posso sair do Brasil quando e como quiser, diz...",o italiano cesare battisti disse nesta segunda...
...,...,...,...
79847,167046,"Apoiado pelos irmãos Gomes, petista toma posse...",engenheiro agronomo e servidor licenciado do i...
79848,167048,"Em cenário de crise, tucano Beto Richa assume ...",o tucano beto richa tinha tudo para comecar se...
79849,167049,Filho supera senador Renan Calheiros e assume ...,o economista renan filho (pmdb) assume nesta q...
79850,167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",destaques da programacao desta quinta-feira (1...


In [13]:
# Drop duplicate rows based on the 'text' column
# This ensures that each text entry in the DataFrame is unique
# The 'inplace=True' parameter modifies the DataFrame in place without returning a new DataFrame
df = df.drop_duplicates(subset=["text"])

print(df.shape)

(79791, 3)


In [14]:
# Calculate descriptive statistics for the length of text in the 'text' column
# The 'str.len()' method computes the length of each string in the 'text' column
# The 'describe()' method provides summary statistics for these lengths
# The list of percentiles [0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.98, 0.99, 0.999] specifies additional percentiles to include in the summary
df["text"].str.len().describe([0.05, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.98, 0.99, 0.999])

count    79791.000000
mean      2673.606898
std       1820.254852
min         28.000000
5%         808.000000
10%       1058.000000
25%       1551.000000
50%       2341.000000
75%       3323.000000
80%       3608.000000
90%       4531.000000
95%       5625.000000
98%       7311.600000
99%       8784.000000
99.9%    17385.680000
max      60816.000000
Name: text, dtype: float64

In [15]:
# Calculate the 2th and 98th percentiles for the length of text in the 'text' column
# The 'quantile' method returns the specified percentiles as a Series
# The 'values' attribute converts the Series to a NumPy array for easier indexing
quantiles = df["text"].str.len().quantile([0.02, 0.98]).values

# Filter the DataFrame to keep only rows where the text length is greater than the 10th percentile
df = df[df["text"].str.len() > quantiles[0]]

# Further filter the DataFrame to keep only rows where the text length is less than or equal to the 99th percentile
df = df[df["text"].str.len() <= quantiles[1]]

print(quantiles)
print(df.shape)

[ 526.  7311.6]
(76597, 3)


In [17]:
# Import the math module for mathematical functions
import math

# Calculate the total number of reviews in the dataset
n = len(df)  # Total number of reviews

# Define the Z-value for a 95% confidence level
# This value corresponds to the number of standard deviations from the mean in a normal distribution
z = 1.96  # Z-value for 95% confidence level

# Define the expected proportion of positive reviews
# We assume 50% (0.5) as the worst-case scenario to maximize the sample size
p = 0.5  # Expected proportion of positive reviews

# Define the margin of error we are willing to accept
# This value represents the maximum acceptable difference between the sample estimate and the true population value
e = 0.05  # Margin of error

# Calculate the required sample size using the formula for sample size estimation
# The formula is derived from the standard error of the proportion
# We use math.ceil to round up to the nearest whole number, ensuring the sample size is sufficient
sample_size = math.ceil((z**2 * p * (1 - p)) / e**2)

# Print the calculated sample size
print(f"Sample size: {sample_size}")

Sample size: 385


In [16]:
# We'll round that to 400 samples for each set.

#  Randomize the dataset to ensure that the samples are shuffled
# 'frac=1.0' means we are shuffling the entire DataFrame
# 'random_state=271828' ensures reproducibility of the shuffling
df = df.sample(frac=1.0, random_state=271828)

# Define the desired sample size for the development and test sets
# We want 400 samples for each set
desired_sample_size = 400

# Split the DataFrame into development/test set and training set
# The first 800 samples (4 * 400) are used for the development and test sets
df_dev_test = df[: desired_sample_size * 4]

# The remaining samples are used for the training set
df_train = df[desired_sample_size * 4 :]

# Print the size of the original dataset
print("Original dataset size: ", len(df))

# Print the size of the training set
print(f"Train set size: {len(df_train)}")

# Print the size of the combined development and test set
print(f"Dev/Test set size: {len(df_dev_test)}")

Original dataset size:  76597
Train set size: 74997
Dev/Test set size: 1600


In [20]:
df_train.to_parquet("./data/train.parquet", index=False)
df_dev_test.to_parquet("./data/dev_test.parquet", index=False)

In [21]:
labeled = pd.read_parquet("data/labeled/noticias_rotuladas_zeroshot.parquet")

labeled.to_csv("data/labeled/labeled.csv", index=False)